In [19]:
import datetime as dt
import numpy as np
import pandas as pd
import scipy.stats as sc
import plotly.express as px

In [20]:
# Need a way to select the date, and the time until expiry
# Need a way to load the data on options available on that date with that expiry based on the strike price
# Need a way to, upon constructing an options portfolio, create the risk graphs
# Can do some additional configuration with stuff like greeks, number of lines to show, etc.

In [21]:
date = dt.datetime(2024, 1, 26)
expiration_time = 17
calls = pd.DataFrame([[4860, 11.20, 11.30, 11.40, 9.65, 20.48]], columns = ['Strike', 'Bid', 'Mid', 'Ask', 'IV', 'Delta'])
model_calls = -10

puts = pd.DataFrame([[4680, 13.70, 13.80, 13.90, 11.58, -20.40]], columns = ['Strike', 'Bid', 'Mid', 'Ask', 'IV', 'Delta'])
model_puts = -10

In [22]:
# Replicating 4860-strike call price using BS

def call_price(r, iv, T, t, K, St):
  d_plus = (np.log (St / K) + (r + iv ** 2 / 2) * (T - t)) / (iv * np.sqrt(T - t))
  d_minus = d_plus - iv * np.sqrt(T - t)
  return sc.norm.cdf(d_plus) * St - sc.norm.cdf(d_minus) * K * np.exp(-r * (T - t))

def put_price(r, iv, T, t, K, St):
  return K * np.exp(-r * (T - t)) - St + call_price(r, iv, T, t, K, St)

In [45]:
S = np.linspace(4620, 4900, 100)
r = 0.03
sigma = 0.1

ts = [0, 6, 12, 16.9999]
portfolio = pd.DataFrame(S, columns = ['x'])
 
for i, t in enumerate(ts):  
  portfolio[str(t)] = -10 * put_price(0.0402, 0.1158, 17/365, t/365, 4680, S) - 10 * call_price(0.0402, 0.0965, 17/365, t/365, 4860, S)

#print(portfolio)
fig = px.line(portfolio, x = 'x', y = portfolio.columns[1:], title = 'PnL vs Stock Price')

fig.show()


              x           0           6          12     16.9999
0   4620.000000 -766.975504 -706.032027 -636.198094 -599.999485
1   4622.828283 -748.555723 -685.971828 -613.257851 -571.716656
2   4625.656566 -730.413897 -666.214288 -590.669673 -543.433828
3   4628.484848 -712.553937 -646.764674 -568.446892 -515.151000
4   4631.313131 -694.979704 -627.628103 -546.602569 -486.868171
..          ...         ...         ...         ...         ...
95  4888.686869 -637.415943 -531.508601 -411.190771 -286.869222
96  4891.515152 -654.999043 -550.211990 -431.712277 -315.152050
97  4894.343434 -672.916680 -569.289362 -452.710383 -343.434879
98  4897.171717 -691.163318 -588.733154 -474.168981 -371.717707
99  4900.000000 -709.733317 -608.535591 -496.071375 -400.000535

[100 rows x 5 columns]
